<h1 align='center'>  Reading Object Class from online images

In [1]:
# Importing needed libraries
import numpy as np
import pandas as pd
import cv2
import time
import urllib

## Defining Yolo weights and config files

In [2]:
"""
Loading YOLO v3 network
"""

with open(r'yolo-coco-data/coco.names') as f:
    labels = [line.strip() for line in f]
        


network = cv2.dnn.readNetFromDarknet('yolo-coco-data/yolov3.cfg',
                                     'yolo-coco-data/yolov3.weights')
layers_names_all = network.getLayerNames()
layers_names_output = \
    [layers_names_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]

probability_minimum = 0.5
threshold = 0.3
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

## TestImage Object classification

In [3]:
url = 'https://aiquire-attachments.s3.ap-south-1.amazonaws.com/2955399654497093%3Af448349112e6f340760099257600608d-frame%203.jpg'
filename = "downloadedImage.jpg"
urllib.request.urlretrieve(url,filename)

('downloadedImage.jpg', <http.client.HTTPMessage at 0x19dc8ebeb08>)

In [4]:
def Get_OutputfromNetwork(image):
    image_BGR = cv2.imread(image)
    h, w = image_BGR.shape[:2]
    blob = cv2.dnn.blobFromImage(image_BGR, 1 / 255.0, (416, 416),
                                 swapRB=True, crop=False)
    network.setInput(blob) 
    output_from_network = network.forward(layers_names_output)
    
    bounding_boxes = []
    confidences = []
    class_numbers = []
    
    for result in output_from_network:
        for detected_objects in result:
            scores = detected_objects[5:]
            class_current = np.argmax(scores)
            confidence_current = scores[class_current]
            
            if confidence_current > probability_minimum:
                box_current = detected_objects[0:4] * np.array([w, h, w, h])                     
                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))           
                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)
            
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences,
                           probability_minimum, threshold)
    
    if len(results) > 0:
        pred = []
        for i in results.flatten():
            pred.append(labels[int(class_numbers[i])])

       
       
    return pred

In [5]:
results = Get_OutputfromNetwork('deepa1.jpg')

In [6]:
results

['person',
 'person',
 'person',
 'person',
 'tvmonitor',
 'diningtable',
 'cup',
 'bowl',
 'chair',
 'apple']

### Creating function to get prediction of Image for xls file

In [7]:
# download's the image and read the text in it.
def Get_ObjClass(url):
    filename = "downloadedImage.jpg"
    
    try:
        urllib.request.urlretrieve(url,filename)
        ObjName = Get_OutputfromNetwork(filename)
        os.remove(filename)
    except:
        empty = ['null']
        return empty
    
    return ObjName

 
<h2 style='color:green'> Prediction for excel sheet of online Images

In [8]:
ad_data = pd.read_csv("ImgAdvertisement_df.csv")

In [9]:
ad_data.head()

,Object ID,Results,Cost Per Result,CTR,Creative Attachment,Link,image_id
0,1,29536,186,1.48,988,https://www.facebook.com/ads/image/?d=AQII31oc...,509
1,2,97570,280,1.90,1125,https://www.facebook.com/ads/image/?d=AQJlmbY5...,1675
2,3,75089,166,1.00,3326,https://aiquire-attachments.s3.ap-south-1.amaz...,176
3,4,92104,111,1.97,725,https://www.facebook.com/ads/image/?d=AQL1me3x...,2647
4,5,60378,429,0.33,344,https://www.facebook.com/ads/image/?d=AQKAE109...,2013


In [10]:
ad_data.shape

(3357, 7)

In [11]:
slice1 = ad_data.head(1000)

In [12]:
slice1.shape

(1000, 7)

In [13]:
import os

In [14]:
def Get_ObjClass(url):
    filename = "downloadedImage.jpg"
    
    
    try:
        urllib.request.urlretrieve(url,filename)
        ObjName = Get_OutputfromNetwork(filename)
        os.remove(filename)
    except:
        empty = ['null']
        return empty
    
    return ObjName

### Predictions

In [15]:
%%time
ad_data["Obj_Names"] = ad_data.Link.apply(Get_ObjClass)

Wall time: 52min 21s


In [17]:
ad_data.head(20)

,Object ID,Results,Cost Per Result,CTR,Creative Attachment,Link,image_id,Obj_Names
0,1,29536,186,1.48,988,https://www.facebook.com/ads/image/?d=AQII31oc...,509,[person]
1,2,97570,280,1.90,1125,https://www.facebook.com/ads/image/?d=AQJlmbY5...,1675,"[laptop, person]"
2,3,75089,166,1.00,3326,https://aiquire-attachments.s3.ap-south-1.amaz...,176,[person]
3,4,92104,111,1.97,725,https://www.facebook.com/ads/image/?d=AQL1me3x...,2647,"[bottle, cup, bottle, bottle, bottle, bottle, ..."
4,5,60378,429,0.33,344,https://www.facebook.com/ads/image/?d=AQKAE109...,2013,"[person, person, person, person, tie, cup, cup..."
5,6,36940,352,0.01,3003,https://www.facebook.com/ads/image/?d=AQKDicA2...,2048,[null]
6,7,31951,337,1.29,2081,https://www.facebook.com/ads/image/?d=AQKSnUeS...,2232,"[person, person, person, person, bowl]"
7,8,98254,335,0.16,2194,https://www.facebook.com/ads/image/?d=AQJJVx8S...,1326,[null]
8,9,10344,452,1.59,481,https://www.facebook.com/ads/image/?d=AQInA5z1...,893,[person]
9,10,76376,439,0.37,532,https://www.facebook.com/ads/image/?d=AQJrYwTU...,1748,[person]


In [18]:
count = 0
for objects in ad_data['Obj_Names']:
    if 'null' in objects:
        count+=1
print(count)

1304


In [19]:
count

1304

In [20]:
ad_data.to_csv('ObjClassPrediction.csv', index=False)

In [42]:
ad_data.columns

Index(['Object ID', 'Results', 'Cost Per Result', 'CTR', 'Creative Attachment',
       'Link', 'image_id', 'Obj_Names'],
      dtype='object')

In [43]:
ad_data = ad_data.drop(['Results', 'Cost Per Result','CTR','Link', 'image_id' ], axis = 1) 
ad_data = ad_data.drop(['Creative Attachment'], axis = 1)

,Object ID,Creative Attachment,Obj_Names
0,1,988,[person]
1,2,1125,"[laptop, person]"
2,3,3326,[person]


In [45]:
ad_data.head(3)

,Object ID,Obj_Names
0,1,[person]
1,2,"[laptop, person]"
2,3,[person]


In [22]:
data = pd.read_excel('Sample Data for Creative Analysis.xlsx')

In [24]:
data.head()

,Object ID,Results,Cost Per Result,CTR,Creative Attachment,Link
0,1,29536,186,1.48,988,https://www.facebook.com/ads/image/?d=AQII31oc...
1,2,97570,280,1.90,1125,https://www.facebook.com/ads/image/?d=AQJlmbY5...
2,3,75089,166,1.00,3326,https://aiquire-attachments.s3.ap-south-1.amaz...
3,4,92104,111,1.97,725,https://www.facebook.com/ads/image/?d=AQL1me3x...
4,5,60378,429,0.33,344,https://www.facebook.com/ads/image/?d=AQKAE109...


In [27]:
ad_data['Creative Attachment'].nunique(), ad_data.shape, data['Creative Attachment'].nunique()

(3357, (3357, 8), 3357)

In [47]:
Full_pred = pd.merge(data, ad_data, how = 'left', on = 'Object ID')

In [48]:
Full_pred.shape, data.shape

((993600, 7), (993600, 6))

In [49]:
Full_pred.head()

,Object ID,Results,Cost Per Result,CTR,Creative Attachment,Link,Obj_Names
0,1,29536,186,1.48,988,https://www.facebook.com/ads/image/?d=AQII31oc...,[person]
1,2,97570,280,1.90,1125,https://www.facebook.com/ads/image/?d=AQJlmbY5...,"[laptop, person]"
2,3,75089,166,1.00,3326,https://aiquire-attachments.s3.ap-south-1.amaz...,[person]
3,4,92104,111,1.97,725,https://www.facebook.com/ads/image/?d=AQL1me3x...,"[bottle, cup, bottle, bottle, bottle, bottle, ..."
4,5,60378,429,0.33,344,https://www.facebook.com/ads/image/?d=AQKAE109...,"[person, person, person, person, tie, cup, cup..."


In [50]:
Full_pred.to_csv('FullPredictions.csv', index=False)